In [4]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Step 2: Load Data
train_data = pd.read_csv("train_multi_output.csv")
test_data = pd.read_csv("test_multi_output.csv")

# Step 3: Select Features and Targets
X_train = train_data[['Season', 'Day_of_the_week', 'DHI', 'DNI', 'GHI','Wind_speed', 'Humidity', 'Temperature']].values
y_train = train_data[['PV_production', 'Wind_production']].values

X_test = test_data[['Season', 'Day_of_the_week', 'DHI', 'DNI', 'GHI','Wind_speed', 'Humidity', 'Temperature']].values
y_test = test_data[['PV_production', 'Wind_production']].values

# Step 4: Scale the data (LSTM performs better with scaled inputs)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Step 5: Reshape input for LSTM [samples, timesteps, features]
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ============================================================
# ⚙️ Step 6: Build LSTM Model with Multiple Activation Functions
# ============================================================

def build_lstm_model(activation_fn='tanh', recurrent_activation_fn='sigmoid', dense_activation='relu'):
    model = Sequential()
    
    # LSTM Layer with customizable activations
    model.add(LSTM(units=64, 
                   activation=activation_fn, 
                   recurrent_activation=recurrent_activation_fn,
                   input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]),
                   return_sequences=False))
    
    # Dropout to prevent overfitting
    model.add(Dropout(0.2))
    
    # Dense Layer with chosen activation
    model.add(Dense(32, activation=dense_activation))
    
    # Output Layer - 2 neurons for PV and Wind production (linear activation for regression)
    model.add(Dense(2, activation='linear'))
    
    model.compile(optimizer='adam', loss='mse')
    return model

# Try different combinations of activations
activation_combos = [
    ('tanh', 'sigmoid', 'relu'),
    ('relu', 'sigmoid', 'relu'),
    ('selu', 'tanh', 'relu'),
    ('tanh', 'hard_sigmoid', 'elu')
]

results = []

# ============================================================
# 🚀 Step 7: Train & Evaluate Different Activation Combinations
# ============================================================
for act_fn, rec_act_fn, dense_act_fn in activation_combos:
    print(f"\nTraining LSTM with activations: LSTM({act_fn} as an Activation Function AND {rec_act_fn} as a Recurrent Activation Function) + Dense({dense_act_fn} as an Activation Function for the Dense Layer)")
    
    model = build_lstm_model(activation_fn=act_fn, 
                             recurrent_activation_fn=rec_act_fn, 
                             dense_activation=dense_act_fn)
    
    early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, min_delta=0.001)
    
    history = model.fit(
        X_train_scaled, y_train_scaled,
        epochs=20,
        batch_size=64,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Predict and inverse transform
    y_pred_scaled = model.predict(X_test_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    
    # Metrics
    mae_pv = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
    mae_wind = mean_absolute_error(y_test[:, 1], y_pred[:, 1])
    mse_pv = mean_squared_error(y_test[:, 0], y_pred[:, 0])
    mse_wind = mean_squared_error(y_test[:, 1], y_pred[:, 1])
    rmse_pv = np.sqrt(mse_pv)
    rmse_wind = np.sqrt(mse_wind)
    r2_pv = r2_score(y_test[:, 0], y_pred[:, 0])
    r2_wind = r2_score(y_test[:, 1], y_pred[:, 1])
    
    # Adjusted R² Calculation
    n = y_test.shape[0]  # number of samples
    p = X_test.shape[1]  # number of predictors (features)
    
    adj_r2_pv = 1 - ((1 - r2_pv) * (n - 1)) / (n - p - 1)
    adj_r2_wind = 1 - ((1 - r2_wind) * (n - 1)) / (n - p - 1)
    
    results.append({
        'LSTM_Activation': act_fn,
        'Recurrent_Activation': rec_act_fn,
        'Dense_Activation': dense_act_fn,
        'PV_R2': r2_pv,
        'Wind_R2': r2_wind,
        'PV_Adj_R2': adj_r2_pv,
        'Wind_Adj_R2': adj_r2_wind,
        'PV_MSE': mse_pv,
        'Wind_MSE': mse_wind,
        'PV_RMSE': rmse_pv,
        'Wind_RMSE': rmse_wind,
        'PV_MAE': mae_pv,
        'Wind_MAE': mae_wind
    })

# ============================================================
# 📊Step 8: Compare Activation Function Combinations and Display in a Single Table
# ============================================================
results_df = pd.DataFrame(results)

# Print the DataFrame as a single table
print("\n~~~~~ Activation Function Comparison ~~~~~")
print(results_df.sort_values(by=['PV_R2', 'Wind_R2'], ascending=False).round(3))




Training LSTM with activations: LSTM(tanh as an Activation Function AND sigmoid as a Recurrent Activation Function) + Dense(relu as an Activation Function for the Dense Layer)
Epoch 1/20


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


341/341 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.0296 - val_loss: 0.0192
Epoch 2/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0196 - val_loss: 0.0184
Epoch 3/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0188 - val_loss: 0.0177
Epoch 4/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0182 - val_loss: 0.0177
Epoch 5/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0177 - val_loss: 0.0174
365/365 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

Training LSTM with activations: LSTM(relu as an Activation Function AND sigmoid as a Recurrent Activation Function) + Dense(relu as an Activation Function for the Dense Layer)
Epoch 1/20


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


341/341 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.0340 - val_loss: 0.0192
Epoch 2/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0200 - val_loss: 0.0180
Epoch 3/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0190 - val_loss: 0.0173
Epoch 4/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0183 - val_loss: 0.0169
Epoch 5/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0177 - val_loss: 0.0166
Epoch 6/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0172 - val_loss: 0.0164
365/365 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

Training LSTM with activations: LSTM(selu as an Activation Function AND tanh as a Recurrent Activation Function) + Dense(relu as an Activation Function for the Dense Layer)
Epoch 1/20


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


341/341 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.0339 - val_loss: 0.0194
Epoch 2/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0194 - val_loss: 0.0180
Epoch 3/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0183 - val_loss: 0.0170
Epoch 4/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0175 - val_loss: 0.0165
Epoch 5/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0170 - val_loss: 0.0163
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

Training LSTM with activations: LSTM(tanh as an Activation Function AND hard_sigmoid as a Recurrent Activation Function) + Dense(elu as an Activation Function for the Dense Layer)


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.0299 - val_loss: 0.0195
Epoch 2/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0205 - val_loss: 0.0189
Epoch 3/20
341/341 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0198 - val_loss: 0.0189
365/365 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

~~~~~ Activation Function Comparison ~~~~~
  LSTM_Activation Recurrent_Activation Dense_Activation  PV_R2  Wind_R2  \
1            relu              sigmoid             relu  0.921    0.535   
2            selu                 tanh             relu  0.916    0.542   
0            tanh              sigmoid             relu  0.911    0.527   
3            tanh         hard_sigmoid              elu  0.902    0.478   

   PV_Adj_R2  Wind_Adj_R2  PV_MSE  Wind_MSE  PV_RMSE  Wind_RMSE  PV_MAE  \
1      0.921        0.535   0.008     0.024    0.092      0.156   0.054   
2      0.916        0.541   0.009     0.024    0.095      0.155   0.060   
0      0.911        0.527   0.010     0.025    0.097    